In [45]:
import os
import numpy as np
from utilityFunctions.utils import *
from utilityFunctions.hinge import get_hinge_features
from utilityFunctions.preprocess import *
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from utilityFunctions.chainCodes import get_chain_code_features
import pickle
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier


In [2]:
#read training images
training_data,labels = load_images_from_folder(r'Training Data')


100%|██████████| 236/236 [00:18<00:00, 12.80it/s]


In [3]:
#preprocess training images
for i in tqdm(range(len(training_data))):
    training_data[i] = preprocess(training_data[i])  

100%|██████████| 236/236 [01:14<00:00,  3.15it/s]


In [4]:
#get features
hinge_feature_vectors = []
for i in range(len(training_data)):
    hinge_feature_vectors.append(get_hinge_features(training_data[i]))


chain_code_feature_vectors = []
for i in range(len(training_data)):
    chain_code_feature_vectors.append(get_chain_code_features(training_data[i])[0])

print(chain_code_feature_vectors[0].shape)
          

(72,)


In [5]:

features = []
for i in range(len(training_data)):
    temp = np.append(hinge_feature_vectors[i], chain_code_feature_vectors[i])
    features.append(temp)



In [6]:
outputFolderTraining = "Training Features"
np.save(os.path.join(outputFolderTraining,
        f"features.npy"), features)
np.save(os.path.join(outputFolderTraining, f"labels.npy"), labels)

In [17]:
#svm model
outputFolderTraining = "Training Features"

features = np.load(os.path.join(
    outputFolderTraining, f"features.npy"))
labels = np.load(os.path.join(outputFolderTraining, f"labels.npy"))

#now we have the features and labels we can train the model

#scale the features
#cs = MinMaxScaler()
cs = StandardScaler()
x = cs.fit_transform(features)
x = np.nan_to_num(x)


#save the scaler
pickle.dump(cs, open(os.path.join(outputFolderTraining, "scaler.pkl"), 'wb'))

clf = SVC(C=10)
clf.fit(x , labels)

pickle.dump(clf, open(os.path.join(outputFolderTraining, "svm.pkl"), 'wb'))


print('SVM model trained and saved')
'''

END OF Feature Extraction and Training

'''

SVM model trained and saved


'\n\nEND OF Feature Extraction and Training\n\n'

In [8]:
#import the model 
clf = pickle.load(open(os.path.join(outputFolderTraining, "svm.pkl"), 'rb'))

#lets test the model on unseen testing data
#read testing images
testing_data, test_labels = load_images_from_folder(r'Testing Data')
#preprocess testing images
for i in range(len(testing_data)):
    testing_data[i] = preprocess(testing_data[i])
#get features
test_hinge_feature_vectors = []
for image in tqdm(testing_data):
    test_hinge_feature_vectors.append(get_hinge_features(image))

test_chain_code_feature_vectors = []
for image in tqdm(testing_data):
    test_chain_code_feature_vectors.append(get_chain_code_features(image)[0])

test_features = []
for i in tqdm(range(len(testing_data))):
    temp = np.append(test_hinge_feature_vectors[i],test_chain_code_feature_vectors[i])
    test_features.append(temp)    

outputFolderTesting = "Testing Features"
np.save(os.path.join(outputFolderTesting,
        f"test_features.npy"), test_features)
np.save(os.path.join(outputFolderTesting, f"test_labels.npy"), test_labels)
        


100%|██████████| 127/127 [00:00<00:00, 126978.93it/s]


In [23]:
outputFolderTesting = "Testing Features"
test_features = np.load(os.path.join(
    outputFolderTesting, f"test_features.npy"))
test_labels = np.load(os.path.join(outputFolderTesting, f"test_labels.npy"))


#load the scaler
sc = pickle.load(open(os.path.join(outputFolderTraining, "scaler.pkl"), 'rb'))
x_test = np.nan_to_num(test_features)
x_test = sc.transform(x_test)


#calculate accuracy
y_pred = clf.predict(x_test)
print('Accuracy:', accuracy_score(test_labels, y_pred)*100, '%')


FileNotFoundError: [Errno 2] No such file or directory: 'Testing Features\\test_features.npy'

In [10]:
testing_data, test_labels = load_images_from_folder(r'Testing Data')
#save test lables to ground_truth.txt in the current directory
with open('ground_truth.txt', 'w') as f:
    for item in test_labels:
        f.write("%s\n" % item)


100%|██████████| 127/127 [00:10<00:00, 11.78it/s]


In [43]:
x = np.nan_to_num(features)

cs = StandardScaler()
x = cs.fit_transform(x)


clf = SVC(C=10)
clf.fit(x, labels)

x_test = np.nan_to_num(test_features)
x_test = cs.transform(x_test)


predicitons = clf.predict(x_test)

#calculate accuracy
print('Accuracy:', accuracy_score(test_labels, predicitons)*100, '%')


Accuracy: 80.31496062992126 %


In [79]:
clf = RandomForestClassifier(n_estimators=210,random_state=0)
clf.fit(features, labels)
predicitons = clf.predict(test_features)
print('Accuracy:', accuracy_score(test_labels, predicitons)*100, '%')


Accuracy: 81.88976377952756 %
